In [3]:
import h5py
import numpy as np
from synthesizer.conversions import lnu_to_absolute_mag
import pandas as pd
import unyt
from unyt import erg, Hz, s
import cmasher as cmr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import sys
sys.path.append("/disk/xray15/aem2/camels/proj2")
from setup_params import get_photometry, get_luminosity_function, get_colour_distribution

# base directories
base_dir = "/disk/xray15/aem2/data/28pams/IllustrisTNG/1P"
input_dir = f"{base_dir}/photometry"

# redshift mappings
redshift_values = {
    '044': {'redshift': 2.00, 'label': 'z2.0'},
    '052': {'redshift': 1.48, 'label': 'z1.5'},
    '060': {'redshift': 1.05, 'label': 'z1.0'},
    '086': {'redshift': 0.10, 'label': 'z0.1'}
}

# LFs
lf_output_dir = {
    "observed": {
        "GALEX": f"{base_dir}/LFs/observed/GALEX"  # Will contain both attenuated/intrinsic
    },
    "restframe": {
        "UV1500": f"{base_dir}/LFs/restframe/UV1500"
    }
}

# colours
colour_output_dir = {
    "observed": {
        "GALEX": f"{base_dir}/colours/observed/GALEX"
    },
    "restframe": {
        "UV1500": f"{base_dir}/colours/restframe/UV1500"
    }
}

# directories including redshift subdirectories
for category in ["observed", "restframe"]:
    for telescope, path in lf_output_dir[category].items():
        for redshift_info in redshift_values.values():
            redshift_dir = os.path.join(path, redshift_info['label'])
            os.makedirs(redshift_dir, exist_ok=True)
            
    for telescope, path in colour_output_dir[category].items():
        for redshift_info in redshift_values.values():
            redshift_dir = os.path.join(path, redshift_info['label'])
            os.makedirs(redshift_dir, exist_ok=True)

# parameters
uvlf_limits = (-24, -16)
uvlf_nbins = 18
colour_limits = (-0.5, 3.5)
nbins_colour = 20

# magnitude limits (DIS) from https://iopscience.iop.org/article/10.1086/520512/pdf
NUV_mag_lim = 24.4
FUV_mag_lim = 24.8

# telescopes and their parameters
telescopes = {
    "GALEX": {
        "category": "observed",
        "types": ["attenuated", "intrinsic"],
        "bands": {
            "FUV": {
                "filter_name": "GALEX FUV",
                "mag_limit": FUV_mag_lim
            },
            "NUV": {
                "filter_name": "GALEX NUV",
                "mag_limit": NUV_mag_lim
            }
        },
        "uvlf_limits": uvlf_limits,
        "uvlf_nbins": uvlf_nbins,
        "colour_pairs": [
            {
                "band1": "FUV",
                "band2": "NUV",
                "limits": colour_limits,
                "nbins": nbins_colour
            }
        ]
    },
    "UV1500": {
        "category": "restframe",
        "type": "restframe",
        "wavelength": "UV1500",
        "bands": {
            "UV1500": {
                "filter_name": "UV1500",
                "mag_limit": None
            }
        },
        "uvlf_limits": uvlf_limits,
        "uvlf_nbins": uvlf_nbins,
        "colour_pairs": []
    }
}

In [ ]:
def process_single_snapshot(sim_name, snap, telescope, telescope_params, spec_type=None):
   """Process a single snapshot for both UVLF and colours"""
   
   if telescope == "UV1500":
       # Special handling for UV1500 (restframe)
       file_path = os.path.join(input_dir, f"IllustrisTNG_{sim_name}_photometry.hdf5")
       with h5py.File(file_path, 'r') as hdf:
           lum_UV1500 = hdf[f'/snap_{snap}/BC03/photometry/luminosity/intrinsic/UV1500'][:]
           lum_UV1500_unyt = lum_UV1500 * erg / (Hz * s)
           abs_mags = lnu_to_absolute_mag(lum_UV1500_unyt)
           
           # Use get_luminosity_function from setup_params
           phi, phi_sigma, hist, bin_lims = get_luminosity_function(
               {"UV1500": abs_mags},  # Create dict to match photo format
               "UV1500",
               *telescope_params["uvlf_limits"],
               n_bins=telescope_params["uvlf_nbins"]
           )
           
           return {
               'uvlf': {
                   'UV1500': (phi, phi_sigma, hist, bin_lims)
               },
               'colours': {}
           }
   
   else:
       # Get list of all filters needed for this telescope (observed)
       filters = [band_info["filter_name"] for band_info in telescope_params["bands"].values()]
       
       # Get photometry with specified spec_type
       photo = get_photometry(
           sim_name=sim_name,
           spec_type=spec_type,
           snap=snap,
           sps="BC03",
           model="IllustrisTNG",
           filters=filters,
           photo_dir=input_dir
       )
       
       # Create masks for each band
       masks = {
           band: photo[band_info["filter_name"]] < band_info["mag_limit"]
           for band, band_info in telescope_params["bands"].items()
       }
       
       # Process UVLFs for each band
       uvlf_results = {}
       for band, band_info in telescope_params["bands"].items():
           phi, phi_sigma, hist, bin_lims = get_luminosity_function(
               photo, 
               band_info["filter_name"], 
               *telescope_params["uvlf_limits"],
               n_bins=telescope_params["uvlf_nbins"]
           )
           uvlf_results[band] = (phi, phi_sigma, hist, bin_lims)
       
       # Process colours for each colour pair
       colour_results = {}
       for colour_pair in telescope_params["colour_pairs"]:
           band1, band2 = colour_pair["band1"], colour_pair["band2"]
           combined_mask = masks[band1] & masks[band2]
           
           colour_dist, bin_lims_colour = get_colour_distribution(
               photo,
               telescope_params["bands"][band1]["filter_name"],
               telescope_params["bands"][band2]["filter_name"],
               *colour_pair["limits"],
               n_bins=colour_pair["nbins"],
               mask=combined_mask
           )
           colour_results[f"{band1}-{band2}"] = (colour_dist, bin_lims_colour)
       
       return {
           'uvlf': uvlf_results,
           'colours': colour_results
       }

# Process each file in the directory
for filename in os.listdir(input_dir):
   if filename.endswith("_photometry.hdf5"):
       sim_name = filename.replace('IllustrisTNG_', '').replace('_photometry.hdf5', '')
       print(f"Processing {sim_name}")
       
       for telescope, telescope_params in telescopes.items():
           print(f"Processing telescope: {telescope}")
           
           # Get category (observed/restframe) and spec_types
           category = telescope_params["category"]
           if telescope == "GALEX":
               spec_types = telescope_params["types"]
           else:
               spec_types = [telescope_params["type"]]
           
           for spec_type in spec_types:
               for snap in redshift_values.keys():
                   try:
                       # Process the snapshot
                       results = process_single_snapshot(sim_name, snap, telescope, 
                                                      telescope_params, spec_type)
                       
                       if telescope == "UV1500":
                           # Save UV1500 (restframe) results
                           for band in results['uvlf']:
                               phi, phi_sigma, hist, bin_lims = results['uvlf'][band]
                               bin_centers = 0.5 * (bin_lims[1:] + bin_lims[:-1])
                               
                               uvlf_df = pd.DataFrame({
                                   'magnitude': bin_centers,
                                   'phi': phi,
                                   'phi_sigma': phi_sigma,
                                   'hist': hist
                               })
                               
                               uvlf_filename = f"UVLF_{sim_name}_UV1500_{redshift_values[snap]['label']}.txt"
                               redshift_dir = os.path.join(lf_output_dir["restframe"]["UV1500"], 
                                                         redshift_values[snap]['label'])
                               uvlf_df.to_csv(
                                   os.path.join(redshift_dir, uvlf_filename), 
                                   index=False, sep='\t'
                               )
                       
                       else:
                           # Save GALEX (observed) UVLF results
                           for band in telescope_params["bands"]:
                               phi, phi_sigma, hist, bin_lims = results['uvlf'][band]
                               bin_centers = 0.5 * (bin_lims[1:] + bin_lims[:-1])
                               
                               uvlf_df = pd.DataFrame({
                                   'magnitude': bin_centers,
                                   'phi': phi,
                                   'phi_sigma': phi_sigma,
                                   'hist': hist
                               })
                               
                               uvlf_filename = f"UVLF_{sim_name}_{band}_{redshift_values[snap]['label']}_{spec_type}.txt"
                               redshift_dir = os.path.join(lf_output_dir["observed"]["GALEX"], 
                                                         redshift_values[snap]['label'])
                               uvlf_df.to_csv(
                                   os.path.join(redshift_dir, uvlf_filename), 
                                   index=False, sep='\t'
                               )
                           
                           # Save GALEX (observed) colour results
                           for colour_pair in telescope_params["colour_pairs"]:
                               colour_name = f"{colour_pair['band1']}-{colour_pair['band2']}"
                               colour_dist, bin_lims_colour = results['colours'][colour_name]
                               bin_centers = 0.5 * (bin_lims_colour[1:] + bin_lims_colour[:-1])
                               
                               colour_df = pd.DataFrame({
                                   'colour': bin_centers,
                                   'distribution': colour_dist
                               })
                               
                               colour_filename = f"Colour_{sim_name}_{colour_name}_{redshift_values[snap]['label']}_{spec_type}.txt"
                               redshift_dir = os.path.join(colour_output_dir["observed"]["GALEX"], 
                                                         redshift_values[snap]['label'])
                               colour_df.to_csv(
                                   os.path.join(redshift_dir, colour_filename), 
                                   index=False, sep='\t'
                               )
                       
                       print(f"Processed {sim_name} {telescope} {spec_type} {redshift_values[snap]['label']}")
                       
                   except Exception as e:
                       print(f"Error processing {sim_name} {telescope} {spec_type} {redshift_values[snap]['label']}: {str(e)}")
                       continue

print("Processing complete!")

Processing 1P_p7_2
Processing telescope: GALEX
Processed 1P_p7_2 GALEX attenuated z2.0
Processed 1P_p7_2 GALEX attenuated z1.5
Processed 1P_p7_2 GALEX attenuated z1.0
Processed 1P_p7_2 GALEX attenuated z0.1
Processed 1P_p7_2 GALEX intrinsic z2.0
Processed 1P_p7_2 GALEX intrinsic z1.5
Processed 1P_p7_2 GALEX intrinsic z1.0
Processed 1P_p7_2 GALEX intrinsic z0.1
Processing telescope: UV1500
Processed 1P_p7_2 UV1500 restframe z2.0
Processed 1P_p7_2 UV1500 restframe z1.5
Processed 1P_p7_2 UV1500 restframe z1.0
Processed 1P_p7_2 UV1500 restframe z0.1
Processing 1P_p9_n1
Processing telescope: GALEX
Processed 1P_p9_n1 GALEX attenuated z2.0
Processed 1P_p9_n1 GALEX attenuated z1.5
Processed 1P_p9_n1 GALEX attenuated z1.0
Processed 1P_p9_n1 GALEX attenuated z0.1
Processed 1P_p9_n1 GALEX intrinsic z2.0
Processed 1P_p9_n1 GALEX intrinsic z1.5
Processed 1P_p9_n1 GALEX intrinsic z1.0
Processed 1P_p9_n1 GALEX intrinsic z0.1
Processing telescope: UV1500
Processed 1P_p9_n1 UV1500 restframe z2.0
Proces